In [4]:
import sys
import os
sys.path.append(os.path.abspath('../../../../src'))

In [5]:
#import SynBPS
from SynBPS.simulation.Memory_process.distributions.Distribution import Distribution
from SynBPS.simulation.Memory_process.distributions.DiscreteDistribution import DiscreteDistribution
from SynBPS.simulation.Memory_process.distributions.ConditionalProbabilityTable import ConditionalProbabilityTable
from SynBPS.simulation.Memory_process.MarkovChain import MarkovChain, cartesian_product, combine_to_list, modify_rules, generate_condprob


In [6]:
!pwd

/Users/mikeriess/Desktop/code repos/SynBPS/src/SynBPS/simulation/Memory_process


In [7]:

def GenerateInitialProb(D=["a","b"], p0_type="regular"):
    import numpy as np
    import pandas as pd
    
    if p0_type == "min_entropy":
        # Example P0 is one-hot
    
        P0 = np.zeros(len(D))
        P0[np.random.randint(0,len(D),1)[0]] = 1
        P0 = P0.tolist()

    if p0_type != "min_entropy":
        
        P0 = []
    
        for d in D:
            #Draw from uniform dist
            x_d = np.random.uniform(0,1,1)[0]
            #print(x_d)
            #Append the value to the vector
            P0.append(x_d)
        
        #Add the p(absorbtion)=0 to P0
        #P0.append(0)
        
        #Normalize
        S_sum = np.sum(P0)
        P0 = P0/S_sum
        
        #Make dataframe
        #P_0_df = pd.DataFrame(P0).T
        #P_0_df.columns = D
        
    return P0#P_0_df


In [8]:
D = ["A","B","C"]
mode = "max_entropy"

# Including absorption state
D_abs = D.copy()
D_abs.append("!")

In [9]:

# Generate the model

#generate initial probabilities
probabilities = GenerateInitialProb(D_abs, p0_type=mode)    
P0 = {}

for i in range(0,len(D_abs)):
    P0.update({D_abs[i]:probabilities[i]})



In [10]:
states = D
h0 = P0
h=2
mode="max_entropy"
n_transitions=5


In [11]:
#####################################
# P1

#for each link
c = cartesian_product(states, states)
d = combine_to_list(c)

#final steps
g = modify_rules(d, states)
p1_input = generate_condprob(g, states, mode, n_transitions)

######################################
# P2

#for each link
c = cartesian_product(states, states)
d = combine_to_list(c)

e = cartesian_product(c, states)
f = combine_to_list(e)

#final steps
g = modify_rules(f, states)
p2_input = generate_condprob(g, states, mode, n_transitions)

In [46]:
seed_value = 1337

######################################
# P1

#for each link
c = cartesian_product(states, states)
d = combine_to_list(c)

#final steps
g = modify_rules(d, states)
p1_input = generate_condprob(g, states, mode, n_transitions, seed_value)

######################################
# P2

#for each link
c = cartesian_product(states, states)
d = combine_to_list(c)

e = cartesian_product(c, states)
f = combine_to_list(e)

#final steps
g = modify_rules(f, states)
p2_input = generate_condprob(g, states, mode, n_transitions, seed_value)

######################################    
# P3

#for each link
c = cartesian_product(states, states)
d = combine_to_list(c)

e = cartesian_product(c, d)
f = combine_to_list(e)

#final steps
g = modify_rules(f, states)
p3_input = generate_condprob(g, states, mode, n_transitions, seed_value)

######################################    
# P4

#for each link
c = cartesian_product(states, states)
d = combine_to_list(c)

e = cartesian_product(c, d)
f = combine_to_list(e)

e = cartesian_product(f, states)
f = combine_to_list(e)

#final steps
g = modify_rules(f, states)
p4_input = generate_condprob(g, states, mode, n_transitions, seed_value)

######################################    
# P5

#for each link
c = cartesian_product(states, states)
d = combine_to_list(c)

e = cartesian_product(c, d)
f = combine_to_list(e)

e = cartesian_product(f, states)
f = combine_to_list(e)

#final steps
g = modify_rules(f, states)
p4_input = generate_condprob(g, states, mode, n_transitions, seed_value)

In [62]:
def recursive_state_process(states, mode, n_transitions, iterations, seed_value=1337):
        def process_iteration(iter_num):
            if iter_num == 1:
                # Base case: P1 process
                c = cartesian_product(states, states)
                d = combine_to_list(c)
                g = modify_rules(d, states)
                return generate_condprob(g, states, mode, n_transitions, seed_value)
            else:
                # Recursive case: build on previous iteration
                prev_result = process_iteration(iter_num - 1)
                prev_states = [item[:-1] for item in prev_result]  # Remove probabilities
                c = cartesian_product(prev_states, states)
                d = combine_to_list(c)
                g = modify_rules(d, states)
                return generate_condprob(g, states, mode, n_transitions, seed_value)
        return process_iteration(iterations)

# Usage
p1_input_2 = recursive_state_process(states, mode, n_transitions, iterations=1)

# Usage
p2_input_2 = recursive_state_process(states, mode, n_transitions, iterations=2)

# Usage
p3_input_2 = recursive_state_process(states, mode, n_transitions, iterations=3)

# Usage
p4_input_2 = recursive_state_process(states, mode, n_transitions, iterations=4)




In [63]:
p1_input_2 == p1_input

True

In [64]:
p2_input_2 == p2_input

True

In [65]:
p3_input_2 == p3_input

True

In [66]:
p4_input_2 == p4_input

True

In [ ]:

         
        p0 = DiscreteDistribution(h0)
         
        p1 = ConditionalProbabilityTable(p1_input, [p0])
         
        p2 = ConditionalProbabilityTable(p2_input, [p1])
         
        p3 = ConditionalProbabilityTable(p3_input, [p2])
         
        p4 = ConditionalProbabilityTable(p4_input, [p3])
         
        HOMC = MarkovChain([p0, p1, p2, p3, p4], random_state=seed_value)

In [124]:
p0 = DiscreteDistribution(h0)
        
p1 = ConditionalProbabilityTable(p1_input, [p0])

p2 = ConditionalProbabilityTable(p2_input, [p1])

HOMC = MarkovChain([p0, p1, p2], random_state=seed_value)

In [125]:
HOMC.distributions[2].parents

In [143]:
HOMC.sample(3)

[np.str_('C'), 'B', 'A']

In [141]:
p0 = DiscreteDistribution(h0)
distributions = [p0]

for order in range(0,h-1):
    if order == 0:
        p_i_input = recursive_state_process(states, mode, n_transitions, iterations=h, seed_value=seed_value)

        p_i = ConditionalProbabilityTable(p_i_input, [p0])
        distributions.append(p_i)
    else:
        p_i_input = recursive_state_process(states, mode, n_transitions, iterations=h, seed_value=seed_value)

        p_i = ConditionalProbabilityTable(p_i_input, distributions[-1])

        distributions.append(p_i)


HOMC = MarkovChain(distributions, random_state=seed_value)

In [144]:
HOMC.sample(3)

[np.str_('A'), 'C', 'B']

In [116]:
distributions[1].parameters
distributions[1].parents

In [117]:
distributions[2].parameters
distributions[2].parents

In [87]:
HOMC.k

2

In [88]:
h

2

In [118]:
HOMC.distributions[0].parameters

[{'A': np.float64(0.42373563115623797),
  'B': np.float64(0.24947749054685903),
  'C': np.float64(0.11526968995146084),
  '!': np.float64(0.21151718834544223)}]

In [119]:
p0.parameters

[{'A': np.float64(0.42373563115623797),
  'B': np.float64(0.24947749054685903),
  'C': np.float64(0.11526968995146084),
  '!': np.float64(0.21151718834544223)}]

In [120]:
HOMC.distributions

In [123]:
HOMC.sample(2)

[np.str_('A'), 'C', 'B']

### Regular functions

In [12]:
P0

{'A': np.float64(0.42373563115623797),
 'B': np.float64(0.24947749054685903),
 'C': np.float64(0.11526968995146084),
 '!': np.float64(0.21151718834544223)}

In [13]:
h0

{'A': np.float64(0.42373563115623797),
 'B': np.float64(0.24947749054685903),
 'C': np.float64(0.11526968995146084),
 '!': np.float64(0.21151718834544223)}

In [14]:
p1_input

[['A', 'A', 0.37494],
 ['A', 'B', 0.22707],
 ['A', 'C', 0.39799],
 ['B', 'A', 0.35367],
 ['B', 'B', 0.24717],
 ['B', 'C', 0.39916],
 ['C', 'A', 0.13291],
 ['C', 'B', 0.49526],
 ['C', 'C', 0.37183]]

In [15]:
p2_input

[['A', 'A', 'A', 0.06603],
 ['A', 'A', 'B', 0.03999],
 ['A', 'A', 'C', 0.07009],
 ['A', 'B', 'A', 0.11574],
 ['A', 'B', 'B', 0.08089],
 ['A', 'B', 'C', 0.13063],
 ['A', 'C', 'A', 0.06601],
 ['A', 'C', 'B', 0.24597],
 ['A', 'C', 'C', 0.18466],
 ['B', 'A', 'A', 0.02491],
 ['B', 'A', 'B', 0.08349],
 ['B', 'A', 'C', 0.13584],
 ['B', 'B', 'A', 0.02703],
 ['B', 'B', 'B', 0.21258],
 ['B', 'B', 'C', 0.09579],
 ['B', 'C', 'A', 0.17065],
 ['B', 'C', 'B', 0.17163],
 ['B', 'C', 'C', 0.07808],
 ['C', 'A', 'A', 0.10408],
 ['C', 'A', 'B', 0.14613],
 ['C', 'A', 'C', 0.19013],
 ['C', 'B', 'A', 0.04697],
 ['C', 'B', 'B', 0.07208],
 ['C', 'B', 'C', 0.16763],
 ['C', 'C', 'A', 0.12497],
 ['C', 'C', 'B', 0.04466],
 ['C', 'C', 'C', 0.10333]]

In [16]:
p0 = DiscreteDistribution(h0)

p1 = ConditionalProbabilityTable(p1_input, [p0])

p2 = ConditionalProbabilityTable(p2_input, [p1])

HOMC = MarkovChain([p0, p1, p2])

In [17]:
HOMC.distributions

In [18]:
HOMC.sample(2)

[np.str_('A'), 'A', 'C']

## Alternate apparoach

In [19]:
def recursive_cartesian_product(states, h):
    if h == 1:
        return [[s] for s in states]
    else:
        prev = recursive_cartesian_product(states, h-1)
        return [p + [s] for p in prev for s in states]

In [22]:
def create_homc(states, h0, h=2, mode="max_entropy", n_transitions=5):
    from SynBPS.simulation.Memory_process.distributions.ConditionalProbabilityTable import ConditionalProbabilityTable
    from SynBPS.simulation.Memory_process.distributions.DiscreteDistribution import DiscreteDistribution
    from SynBPS.simulation.Memory_process.MarkovChain import MarkovChain, combine_to_list, modify_rules, generate_condprob

    # Generate all conditional probability tables
    cpt_inputs = []
    for i in range(1, h+1):
        d = recursive_cartesian_product(states, i)
        g = modify_rules(d, states)
        cpt_input = generate_condprob(g, states, mode, n_transitions)
        cpt_inputs.append(cpt_input)

    # Create the Markov Chain
    p0 = DiscreteDistribution(h0)
    distributions = [p0]

    for i in range(h):
        if i == 0:
            p = ConditionalProbabilityTable(cpt_inputs[i], [p0])
        else:
            p = ConditionalProbabilityTable(cpt_inputs[i], [distributions[-1]])
        distributions.append(p)

    HOMC = MarkovChain(distributions)

    return HOMC


In [23]:
homc = create_homc(states, h0, h=2, mode="max_entropy", n_transitions=5)
homc.sample(4)

[np.str_('A'), 'A', 'A', 'B']

In [82]:
homc.k

2

In [83]:
homc.distributions

In [84]:
homc.distributions[0].parameters

[{'A': np.float64(0.3713075296921806),
  'B': np.float64(0.09158428623363077),
  'C': np.float64(0.3231436014373088),
  '!': np.float64(0.21396458263687976)}]

In [91]:
homc = create_homc(states, h0, h=6, mode="max_entropy", n_transitions=5)
homc.sample(8)

IndexError: index 0 is out of bounds for axis 0 with size 0

# Python functions

In [21]:
from distributions.DiscreteDistribution import DiscreteDistribution
d1 = DiscreteDistribution({'A': 0.25, 'B': 0.75})
d1

In [22]:
from distributions.ConditionalProbabilityTable import ConditionalProbabilityTable

d2 = ConditionalProbabilityTable([['A', 'A', 0.1],
                                      ['A', 'B', 0.9],
                                      ['B', 'A', 0.6],
                                      ['B', 'B', 0.4]], [d1])

d3 = ConditionalProbabilityTable([['A', 'A', 'A', 0.4],
                                      ['A', 'A', 'B', 0.6],
                                      ['A', 'B', 'A', 0.8],
                                      ['A', 'B', 'B', 0.2],
                                      ['B', 'A', 'A', 0.9],
                                      ['B', 'A', 'B', 0.1],
                                      ['B', 'B', 'A', 0.2],
                                      ['B', 'B', 'B', 0.8]], [d1, d2])

d2

In [23]:
from MarkovChain import MarkovChain

In [24]:
model = MarkovChain([d1, d2, d3])
model

In [25]:
#print(model.to_json())

In [26]:
model.distributions

In [27]:
model.summarize(sequences=["A","A"])

In [28]:
model.k

2

In [29]:
model.sample

<bound method MarkovChain.sample of <MarkovChain.MarkovChain object at 0x107042a50>>

In [30]:
model.sample(5)

['B', 'A', 'A', 'B', 'A']